In [470]:
import pandas as pd
import numpy as np

#VARIABLES AND DATA
candles = pd.read_csv("XBTUSD-1m-data.csv")
candle = pd.Series()
    
def candleseries(candleamount):
    # iterate over rows with iterrows()
    for index, data in candles.head(candleamount).iterrows():
        #determine if candles are of opposite type
        if data['open'] < data['close']:
            type = "green"
        elif data['open'] > data['close']:
            type = "red"
        else:
            type = "abs_doji"
     #append size
        size = [data['open'], data['close'], abs(data['open']-data['close']), type]
        candle.at[index] = size

#realtime func
def tradesignal(i = -1, candleamount = 500, threshold = 1, ignoredoji = False):
    if candle.iloc[i][2] == candle.iloc[i-1][2]:
        return("WAIT")
    elif (candle.iloc[i][2] * threshold) > (candle.iloc[i-1][2]) and (ignoredoji == False or candle.iloc[i-1][2] > 1):
        if candle.iloc[i][3] == "red":
            return("SELL")
        elif candle.iloc[i][3] == "green":
            return("BUY")
        else: return("WAIT")
    else:
        return("WAIT")

#back-test on the series extrapolated from price data
def backtest(candleamount, threshold=1, ignoredoji=False):
    #first generate a candle-series!
    candleseries(candleamount)
    #generate index series for all candles except for the oldest one (no prev candle for it!)
    cseries = iter(candle.index[::-1])
    next(cseries)
    #list to store signals in for easier display
    signals=[]
    #generate a trade signal for every candle except for the last, and store in the list we created
    for i in cseries:
        i = -1*(i)
        signals.append(tradesignal(i, candleamount, threshold, ignoredoji))
    return(signals)

#run the back-test function
#store into data-frame to compare w/ different engulf parameters
#D=0: including DOJIs in back-test data
#D=1: excluding DOJIs in back-test data
#D=2: both inc and exc in back-test data
def testdata(candleamount=500, thresholds=[.1,.25,.5, 1], D=2):
    btdata = pd.DataFrame()
    if D==0 or D==2:    
        btdata[candleamount, thresholds[0]] = backtest(50, .1)
        btdata[candleamount, threshold[1]] = backtest(50, .25)
        btdata[candleamount, threshold[2]] = backtest(50, .5)
        btdata[candleamount, threshold[3]] = backtest(50, 1)
    if D==1 or D==2:
        btdata[candleamount, threshold[0]] = backtest(50, .1, True)
        btdata[candleamount, threshold[1]] = backtest(50, .25, True)
        btdata[candleamount, threshold[2]] = backtest(50, .5, True)
        btdata[candleamount, threshold[3]] = backtest(50, 1, True)
    print("Hey")
    print(btdata)

#
testdata()

C:\Users\eyese\.julia\conda\3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


NameError: name 'threshold' is not defined